In [3]:
import pandas as pd
from pandas import IndexSlice as idx

# load the dataset
dataset = pd.read_hdf('/home/groovyjac/projects/autonomous-portfolio-management/main_data_store_JDKv1.h5', 'stocks/prices/daily')
        
# select a subset of columns
dataset = dataset.loc[idx[:, '2005-02-15':'2023-02-15'], ['adjusted_close', 'volume', 'RSI', 'MACD', 'MACD_signal', 'MACD_hist', 'BB_upper', 'BB_middle', 'BB_lower']]
        
# assign names to the index
dataset.index.names = ['ticker', 'date']

In [5]:
#dataset.head()
dataset.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2121349 entries, ('AAPL', '2005-02-15') to ('NWS', '2023-02-15')
Data columns (total 9 columns):
 #   Column          Dtype  
---  ------          -----  
 0   adjusted_close  float64
 1   volume          int64  
 2   RSI             float64
 3   MACD            float64
 4   MACD_signal     float64
 5   MACD_hist       float64
 6   BB_upper        float64
 7   BB_middle       float64
 8   BB_lower        float64
dtypes: float64(8), int64(1)
memory usage: 154.4+ MB


In [33]:
import backtrader as bt
import pandas as pd
from pandas import IndexSlice as idx

class CustomPandasData(bt.feeds.PandasData):
    lines = ('rsi', 'macd', 'macd_signal', 'macd_hist', 'bb_upper', 'bb_middle', 'bb_lower')
    params = (
        ('datetime', 'date'),
        ('open', -1),
        ('high', -1),
        ('low', -1),
        ('close', 'adjusted_close'),
        ('volume', 'volume'),
        ('openinterest', None),
        ('rsi', 'RSI'),
        ('macd', 'MACD'),
        ('macd_signal', 'MACD_signal'),
        ('macd_hist', 'MACD_hist'),
        ('bb_upper', 'BB_upper'),
        ('bb_middle', 'BB_middle'),
        ('bb_lower', 'BB_lower')
    )

# Load your dataset
dataset = pd.read_hdf('/home/groovyjac/projects/autonomous-portfolio-management/main_data_store_JDKv1.h5', 'stocks/prices/daily')
dataset = dataset.loc[idx[:, '2005-02-15':'2023-02-15'], ['adjusted_close', 'volume', 'RSI', 'MACD', 'MACD_signal', 'MACD_hist', 'BB_upper', 'BB_middle', 'BB_lower']]
dataset.index.names = ['ticker', 'date']

# Reset the index
dataset = dataset.reset_index()

# Convert the 'date' column to datetime
dataset['date'] = pd.to_datetime(dataset['date'])

class AdvancedTechnicalStrategy(bt.Strategy):
    params = (
        ('rsi_entry', 30),
        ('rsi_exit', 70),
        ('macd_short', 12),
        ('macd_long', 26),
        ('macd_signal', 9),
        ('bb_period', 20),
        ('bb_deviation', 2)
    )

    def __init__(self):
        self.rsi = bt.indicators.RelativeStrengthIndex(period=self.params.rsi_entry)
        self.macd = bt.indicators.MACDHisto(
            period_me1=self.params.macd_short,
            period_me2=self.params.macd_long,
            period_signal=self.params.macd_signal
        )
        self.bollinger = bt.indicators.BollingerBands(
            period=self.params.bb_period,
            devfactor=self.params.bb_deviation
        )

    def next(self):
        if not self.position:
            if self.rsi < self.params.rsi_entry and self.macd.histo > 0:
                self.buy()
        else:
            if self.rsi > self.params.rsi_exit or self.macd.histo < 0:
                self.close()

if __name__ == '__main__':
    cerebro = bt.Cerebro()

    # Use the provided dataset
    data = CustomPandasData(dataname=dataset[dataset['ticker'] == 'AAPL'])

    # Add data to Cerebro
    cerebro.adddata(data)

    # Add strategy
    cerebro.addstrategy(AdvancedTechnicalStrategy)

    # Set up the initial cash
    cerebro.broker.setcash(10000.0)

    # Set the commission
    cerebro.broker.setcommission(commission=0.001)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run the backtest
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Plot the result
    #cerebro.plot()

Starting Portfolio Value: 10000.00
Final Portfolio Value: 10000.00


In [19]:
# Load your dataset
dataset = pd.read_hdf('/home/groovyjac/projects/autonomous-portfolio-management/main_data_store_JDKv1.h5', 'stocks/prices/daily')
dataset = dataset.loc[idx[:, '2005-02-15':'2023-02-15'], ['adjusted_close', 'volume', 'RSI', 'MACD', 'MACD_signal', 'MACD_hist', 'BB_upper', 'BB_middle', 'BB_lower']]
dataset.index.names = ['ticker', 'date']

# Reset the index
dataset = dataset.reset_index()

dataset[dataset['ticker'] == 'AAPL'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4532 entries, 0 to 4531
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ticker          4532 non-null   object 
 1   date            4532 non-null   object 
 2   adjusted_close  4532 non-null   float64
 3   volume          4532 non-null   int64  
 4   RSI             4532 non-null   float64
 5   MACD            4532 non-null   float64
 6   MACD_signal     4532 non-null   float64
 7   MACD_hist       4532 non-null   float64
 8   BB_upper        4532 non-null   float64
 9   BB_middle       4532 non-null   float64
 10  BB_lower        4532 non-null   float64
dtypes: float64(8), int64(1), object(2)
memory usage: 424.9+ KB


In [ ]:
# Load and preprocess the dataset
preprocessing = Preprocessing(dataset_path='/home/groovyjac/projects/autonomous-portfolio-management/main_data_store_JDKv1.h5')
dataset = preprocessing.load_dataset()
returns = preprocessing.select_most_traded_stocks(dataset)
T = 52  # weeks
data = preprocessing.create_sliding_window(returns, T)

lstm_model = LSTM_model(data)
X_train, y_train, X_test, y_test = lstm_model.prepare_data()

# Load the trained LSTM model from the saved .h5 file
lstm_model = load_model('results/lstm_model/lstm_regressionv2.h5')

# Assuming you have prepared X_test (input features) and y_test (true labels) for evaluation
# Make predictions on the test dataset
test_predictions = lstm_model.predict(X_test).squeeze()

# Evaluate the model performance (Mean Squared Error) on the test dataset
mse = lstm_model.evaluate(X_test, y_test, verbose=0)

In [32]:
import pandas as pd
from pandas import IndexSlice as idx
import backtrader as bt

# Load the dataset
dataset = pd.read_hdf('/home/groovyjac/projects/autonomous-portfolio-management/main_data_store_JDKv1.h5', 'stocks/prices/daily')

# Select a subset of columns
dataset = dataset.loc[idx[:, '2005-02-15':'2023-02-15'], ['adjusted_close', 'volume']]

# Assign names to the index
dataset.index.names = ['ticker', 'date']

# Reset the index
dataset = dataset.reset_index()

dataset['date'] = pd.to_datetime(dataset['date'])
dataset['date'] = dataset['date'].apply(lambda x: x.to_pydatetime().date())

class CustomStrategy(bt.Strategy):
    params = (
        ('rsi_period', 14),
        ('macd_short', 12),
        ('macd_long', 26),
        ('macd_signal', 9),
        ('bbands_period', 20),
        ('bbands_devfactor', 2),
    )

    def __init__(self):
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=self.params.rsi_period)
        self.macd = bt.indicators.MACD(self.data.close, period_me1=self.params.macd_short, period_me2=self.params.macd_long, period_signal=self.params.macd_signal)
        self.bbands = bt.indicators.BollingerBands(self.data.close, period=self.params.bbands_period, devfactor=self.params.bbands_devfactor)

    def next(self):
        current_day_of_week = self.data.datetime.date().weekday()
        previous_day_of_week = self.data.datetime.date(-1).weekday()

        # Execute trades only once a week
        if current_day_of_week < previous_day_of_week:
            # Buy conditions
            if (
                self.rsi < 30 and
                self.macd.macd > self.macd.signal and
                self.data.close < self.bbands.lines.bot
            ):
                self.buy()

            # Sell conditions
            elif (
                self.rsi > 70 and
                self.macd.macd < self.macd.signal and
                self.data.close > self.bbands.lines.top
            ):
                self.sell()

# Initialize the Cerebro engine
cerebro = bt.Cerebro()

# Add the custom strategy to the Cerebro engine
cerebro.addstrategy(CustomStrategy)

# Create a data feed from the dataset
data = bt.feeds.PandasData(dataname=dataset[dataset['ticker'] == 'AAPL'])

# Add the data feed to the Cerebro engine
cerebro.adddata(data)

# Set the initial cash balance
cerebro.broker.setcash(10000.0)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run the backtest
cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Plot the result
cerebro.plot()

Starting Portfolio Value: 10000.00


AttributeError: 'numpy.int64' object has no attribute 'to_pydatetime'